# 정형 데이터와 비정형 데이터
정형 데이터(머신러닝에 잘 맞음): 어떤 구조로 가지런히 정리되어 있는 데이터. csv나 데이터베이스, 엑셀에 저장하기 쉬움<br>
비정형 데이터(딥러닝에 잘 맞음): 데이터 베이스나 엑셀로 표현하기 어려운 것들(텍스트 데이터, 사진, 음악)

# 앙상블 학습
정형 데이터를 다루는 가장 좋은 알고리즘
ex) 랜덤 포레스트

#랜덤 포레스트(Random Forest)
앙상블 학습의 대표 주자 중 하나<br>
결정 트리를 랜덤하게 만들어서 결정 트리의 숲을 형성하는 알고리즘<br>
각 트리를 훈련하기 위한 데이터를 랜덤하게 뽑는데 이 데이터를 만드는 법이 독특하다.<br>
각 노드를 분할할 때, 전체 특성 중에서 일부 특성을 무작위로 고른 다음 이중에서 최선의 분할을 찾는다.<br>
RandomForestClassifier는 기본적으로 

---
ex) 트리 훈련 데이터<br>
예를 들어 1000개의 샘플이 있고 중복을 허용하여 1000개의 샘플을 뽑는다. 이것을 ***부트스트랩 샘플***이라고 한다.


In [78]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target= wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [79]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs = -1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
# 훈련 데이터 점수, 검증 데이터 점수
print(np.mean(score['train_score']), np.mean(score['test_score']))

0.9973541965122431 0.8905151032797809


In [80]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


RandomForestClassifier는 자체적으로 모델을 평가하는 점수를 얻을 수 있다.<br>
교차 검증을 대신 할 수 있다.<br>
부트스트랩 샘플에 포함되지 않은 샘플을 OOB(out of bag)이라고 한다.<br>
이 OOB로 bootstrap sample로 훈련한 결정 트리를 평가할 수 있다.<br>
방법은 RandomForestClassfier 클래스의 oob_score 매개변수를 True로 지정한다.<br>
각 결정 트리의 oob점수를 평균하여 출력한다. (RandomForestClassifier.oob_score_)

In [81]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


# 엑스트라 트리(Extra Trees)
기본적으로 랜덤포레스트와 동일. 하지만 부트스트랩 샘플을 사용하지 않는다.<br>
노드를 분할할 때 가장 좋은 분할을 찾는 것이 아닌 무작위로 분할한다.

In [82]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [83]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


#그레이디언트 부스팅
깊이가 얕은 결정트리를 사용해서 이전트리의 오차를 보완하는 방법<br>
깊이가 max_depth인 트리를 순차적으로 이전 트리를 보완하며 n_estimators개 만듬.<br>
일반적으로 랜덤포레스트보다 조금 더 높은 성능을 얻을 수 있다. 훈련속도가 느리다.(순서대로 한개의 트리만 만들기 때문에 n_jobs 매개변수가 없다.)

---
경사하강법을 사용하여 트리를 앙상블에 추가함.<br>
분류: 로지스틱 손실 함수<br>
회귀: 평균 제곱 오차 함수<br>
트리를 경사하강법의 샘플로 적용(learning_rate: 트리가 앙상블에 기여하는 정도(산을 내려가는 속도))<br>

---
subsample 매개변수는 1.0으로 전체 훈련 세트를 사용한다. (배치 경사 하강법)<br>
1보다 작을 경우 (확률적, 미니배치 경사 하강법)

In [84]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [85]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2,  random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [86]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)
print(gb.score(test_input, test_target))

[0.15872278 0.68010884 0.16116839]
0.8707692307692307


#히스토그램 기반 그레이디언트 부스팅(HistGradientBoostingClassifier)
입력 특성을 256개의 구간으로 나눈다.<br>
255 + 1 <- 어떤 샘플에 누락된 값이 있다면 때어 놓은 한 개로 판단한다.

In [87]:
# from sklearn.experimental import enable_hist_gradient_boosting 현재 버전에는 안정적이게 histogram gradient boosting을 사용할 수 있기 때문에 사용하지 않아도 된다.
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


### 히스토그램 기반 그레이디언트 부스팅의 특성 중요도
특성 중요도를 계산하기 위해서는 permutation_importance() 함수를 이용하는데 이 함수는 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지를 관찰해서 어떤 특성이 중요한지 계산한다. n_repeats 매개변수는 랜덤하게 섞을 횟수를 지정

In [88]:
from sklearn.inspection import permutation_importance
hgb.fit(train_input, train_target)
# train
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances)  # 특성 중요도 0: alcohol 특성을 10번 랜덤으로 섞어 나온 평균들, 1: sugar, 2: pH
print(result.importances_mean)  # 중요도 평균 0: alcohol, 1: sugar, 2: pH

[[0.08793535 0.08350972 0.08908986 0.08312488 0.09274581 0.08755051
  0.08601116 0.09601693 0.09082163 0.09082163]
 [0.22782374 0.23590533 0.23936887 0.23436598 0.23725226 0.23436598
  0.23359631 0.23398114 0.23994612 0.22724649]
 [0.08581874 0.08601116 0.08062344 0.07504329 0.08427939 0.07792957
  0.07234943 0.07465846 0.08139311 0.08466423]]
[0.08876275 0.23438522 0.08027708]


In [89]:
# test
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [90]:
# hgb 성능 최종 확인
print(hgb.score(test_input, test_target))

0.8723076923076923


#XGBoost
GBM에 비하여 Optimization, algorithmic 향상<br>


1.   System Optimization
<br>*   Tree pruning using depth-first approch(criterion 대신 max_dept을 정해줌. depth-first가 계산 상의 이득을 가져옴)
<br>*   병렬처리를 효과적으로 하기 위해서 학습 모델 구조 변경(computing overheads상쇄)
<br>*   하드웨어 최적화
2.   Algorithmic Enhancements
<br>*   Regularization: L1, L2 규제 부여
<br>*   Spasity Awareness: Sparity를 인정하고 다양한 유형의 Sparse pattern 처리
<br>*   Weight Quantile Sketh: "distributed weighted Quantile Sketch" 알고리즘을 사용하여 최적의 분기점을 효율적으로 찾아냄
 



In [91]:
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8824322471423747 0.8726214185237284


# LightGBM
마이크로소프트에서 만든 모델.<br>
main 기술: 
  1. GOSS: Gradient-based One-Side Sampling (데이터 개체 수를 줄임)<br>
  2. EFB: Exclusive Feature Bundling(Feature 수를 줄임)

---
***GOSS***(XGBoost에 비해서 속도와 성능 특히 속도를 향상시켰다.)<br>

GBDT모델에서 가중치(데이터 개체 중요도)를 의미 하는 것은 gradient (데이터의 기울기가 작다는 것은 훈련 오차가 작다는 것을 의미)<br>
GOSS는 큰 gradient 갖는 데이터는 남겨두고 작은 gradient를 갖는 데이터들을 무작위 sampling한다.

---
1. 데이터 개체들의 Gradient의 절대값에 따라 데이터 개체들을 정렬함<br>
2. 상위 100a 개의 개체를 추출함(a:ratio of large gradient data)<br>
3. 나머지 개체들 집합에서 100b개의 개체를 무작위로 추출함(b:ratio of small gradient data)<br>
4. 정보 획득을 계산할 때, 위의 2-3 과정을 통해 추출된 Sampled Data를 상수( $\frac{1- a}{b}$ )를 이용하여 증폭시킴<br>

---
***EFB***<br>
희소한 변수 공간의 특성에 따라 배타적인 변수들을 하나의 변수로 묶는 걸 EFB라고 한다.<br>

In [92]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9338079582727165 0.8789710890649293
